In [1]:
import numpy as np
from mxnet import nd
import mxnet as mx
classes = ['background','p1','p2','p3','p4',
           'p5','p6','p7','p8','p9','p10',
           'p11','p12','p13','p14','p15',
           'p16','p17','p18','p19']
# RGB color for each class
colormap = [[0,0,0],[128,0,0],[0,128,0], [128,128,0], [0,0,128],
            [128,0,128],[0,64,128],[128,128,128],[64,0,0],[192,0,0],
            [64,128,0],[192,128,0],[64,0,128],[192,0,128],
            [64,128,128],[192,128,128],[0,64,0],[128,64,0],
            [0,192,0],[128,192,0]]

min_coordinate = [[504,697],[1063,606],[948,888],[286,902],[1079,1191],[1053,1507],[1048,1661],
                  [993,1729],[1025,1706],[415,1391],[1121,1355],[1138,1388],[1245,1294],
                  [1212,1477],[1170,1152],[1114,1756],[635,1090],[1081,1100],[350,1006]]
expand_size = 55
landmark_index = 9
# len(classes), len(colormap)



In [2]:
from mxnet import image
import sys

sys.path.append('..')
import utils
from time import time

data_root = '../data'
image_root = data_root + '/CephalometricLandmark/CroppedImage'
# image_root = data_root + '/CephalometricLandmark/ContrastImage1'
txt_root = data_root + '/CephalometricLandmark/AnnotationsByMD'


def read_images(dataset_num=0):
    contrast_size = 0
    if dataset_num == 0:
        begin_index = 1
        end_index = 151
        contrast_size = 0
    elif dataset_num == 1:
        begin_index = 151
        end_index = 301
    else:
        begin_index = 301
        end_index = 401

    data  = [None] * (end_index - begin_index) * (contrast_size + 1)
    label = [None] * (end_index - begin_index) * (contrast_size + 1)
    label_int = [None] * (end_index - begin_index) * (contrast_size + 1)
    #     expand_label =  [None] * (end_index - begin_index)
    #     bounding_box = [[[2000,2000],[0,0],[0,0]] for i in range(19)]
    index = 0
    for contrast_index in range(contrast_size + 1):
#         image_root = data_root + '/CephalometricLandmark/ContrastImage' + str(contrast_index)
#         image_root = data_root + '/CephalometricLandmark/GaussianNoise' + str(contrast_index)
        for i in range(begin_index, end_index):
            image_filename = image_root + "/%02d/%03d.bmp" % (landmark_index + 1, i)
            txt_filename1 = txt_root + '/400_senior' + "/%03d.txt" % i
            txt_filename2 = txt_root + '/400_junior' + "/%03d.txt" % i
            # #         label_image[index] = nd.zeros_like(data[index])

            with open(txt_filename1, 'r') as f:
                txts = f.read().split()
            with open(txt_filename2, 'r') as f:
                txts1 = f.read().split()
            x = int((int(txts[landmark_index].split(',')[0]) + int(txts1[landmark_index].split(',')[0]))/2) - min_coordinate[landmark_index][0]
            y = int((int(txts[landmark_index].split(',')[1]) + int(txts1[landmark_index].split(',')[1]))/2) - min_coordinate[landmark_index][1]
            #         label_image[index][y-expand_size:y+expand_size,x-expand_size:x+expand_size] = colormap[landmark_index+1]
#             x = int(txts1[landmark_index].split(',')[0])- min_coordinate[landmark_index][0]
#             y = int(txts1[landmark_index].split(',')[1])- min_coordinate[landmark_index][1]
            minx = x - expand_size
            maxx = x + expand_size
            if minx < 0:
                minx = 0
            if maxx >= 640:
                maxx = 639

            miny = y - expand_size
            maxy = y + expand_size
            if miny < 0:
                miny = 0
            if maxy >= 640:
                maxy = 639
            data[index] = image.imread(image_filename)
            label[index] = nd.zeros((data[index].shape[0], data[index].shape[1]))
            label[index][miny:maxy, minx:maxx] = 1
            label_int[index] = [np.float32(x),np.float32(y)]
#             label[index] = nd.flip(label[index],0)
#             print(data[index].shape,label[index].shape)
            index += 1
            
    return data, label_int

# train_images, train_label_images, train_labels,bounding_boxes= read_images(0)


In [3]:
# imgs = [train_images[0],train_label_images[0]]
# # print(bounding_boxes)
# # print(type(train_images))
# landmarkindex = 2
# margin_size = 100
# for i in range(19):
# #     imgs += [train_images[i][900:1100,800:1000],train_labels[i][990:1050,800:860]]
# #     x = train_labels[i][1][0]
# #     y = train_labels[i][1][1]
# #     size = 40
# #     imgs += [train_images[i][y-size:y+size,x-size:x+size],train_label_images[i][y-size:y+size,x-size:x+size]]
    
#     minx = bounding_boxes[0][0]
#     miny = bounding_boxes[0][1]
#     maxx = bounding_boxes[1][0]
#     maxy = bounding_boxes[1][1]

#     imgs += [train_images[i][miny-margin_size:maxy+margin_size,minx-margin_size:maxx+margin_size],train_label_images[i][miny-margin_size:maxy+margin_size,minx-margin_size:maxx+margin_size]]
# # print(train_labels[0][981:1001,819:839])
# utils.show_images(imgs, nrows=19, ncols=2,figsize=(12,160))
# [im.shape for im in imgs]

In [4]:

# imgs = []
# landmark_index = 0
# for i in range(10):
#     crop_image = image_crop(train_images[i],bounding_boxes[0],bounding_boxes[1])
#     crop_label = image_crop(train_label_images[i],bounding_boxes[0],bounding_boxes[1])
#     imgs += [crop_image,crop_label]
# utils.show_images(imgs, nrows=10, ncols=2,figsize=(12,160))

In [5]:
from mxnet import gluon
from mxnet import nd
import random

rgb_mean = nd.array([0.485, 0.456, 0.406])
rgb_std = nd.array([0.229, 0.224, 0.225])

def normalize_image(data):
#     mx.random.seed(np.random.randint(1, 1000))
#     noise = mx.nd.random.normal(0, 1, shape=(640, 640, 1), dtype=np.float32)
#     return ((data.astype('float32')+ 10*noise).clip(0, 255) / 255 - rgb_mean) / rgb_std
    return (data.astype('float32') / 255 - rgb_mean) / rgb_std

class VOCSegDataset(gluon.data.Dataset):

    def __init__(self, dataset_num, crop_size):
        self.crop_size = crop_size
        self.data, self.label = read_images(dataset_num=dataset_num)
        self.data[:] = [normalize_image(im) for im in self.data]
#         for i in range(len(self.data)):
#             tmp = image_crop(self.data[i], self.bounding_box[0], self.bounding_box[1],*self.crop_size)
#             self.data[i].reshape(tmp.shape)
#             self.data[i] = tmp.transpose((2,0,1))
#             tmp = image_crop(self.label_image[i], self.bounding_box[0], self.bounding_box[1],*self.crop_size)
#             self.label_image[i].reshape(tmp.shape)
#             self.label_image[i] = tmp.transpose((2,0,1))
#         print('Read '+str(len(self.data))+' examples')

    def __getitem__(self, idx):
#         data,label = image_crop(self.data[idx],self.label[idx],self.bounding_box[0], self.bounding_box[1],*self.crop_size)
        data = self.data[idx]
        label = self.label[idx]
        
#         aug1 = image.HorizontalFlipAug(1)
# #         aug2 = image.BrightnessJitterAug(1)
# #         aug3 = image.ContrastJitterAug(1)
#         if random.random() > 0.5:
#         print(data.shape)
#         data =  nd.flip(data, axis=0)
#         label = nd.flip(label,0)
#         data = aug2(data)
#         data = aug3(data)
            
        return data.transpose((2,0,1)), label

    def __len__(self):
        return len(self.data)

In [6]:
input_shape = (640, 640)
voc_train = VOCSegDataset(0, input_shape)
voc_test1 = VOCSegDataset(1, input_shape)
# voc_test2 = VOCSegDataset(2, input_shape)

In [7]:
batch_size = 4
train_data = gluon.data.DataLoader(
    voc_train, batch_size, shuffle=True,last_batch='discard')
test_data = gluon.data.DataLoader(
    voc_test1, batch_size,last_batch='discard')

for data, label in train_data:
    print(data.shape)
    print(label.shape)
    break

(4, 3, 640, 640)
(4, 2)


In [8]:
from mxnet.gluon import nn
from mxnet import nd

conv = nn.Conv2D(10, kernel_size=4, padding=1, strides=2)
conv_trans = nn.Conv2DTranspose(3, kernel_size=4, padding=1, strides=2)

conv.initialize()
conv_trans.initialize()

x = nd.random.uniform(shape=(1,3,64,64))
y = conv(x)
print('Input:', x.shape)
print('After conv:', y.shape)
print('After transposed conv', conv_trans(y).shape)

Input: (1, 3, 64, 64)
After conv: (1, 10, 32, 32)
After transposed conv (1, 3, 64, 64)


In [9]:
from mxnet.gluon.model_zoo import vision as models
pretrained_net = models.resnet50_v2(pretrained=True)

# (pretrained_net.features[-4:], pretrained_net.output)

In [10]:
net = nn.HybridSequential()
for layer in pretrained_net.features[:-2]:
    net.add(layer)

# x = nd.random.uniform(shape=(1,3,*input_shape))
# print('Input:', x.shape)
# print('Output:', net(x).shape)

In [11]:
num_classes = len(classes)

with net.name_scope():
    net.add(
        nn.Dense(4096),
        nn.Dense(2)
    )
# print(net)

In [12]:
def bilinear_kernel(in_channels, out_channels, kernel_size):
    factor = (kernel_size + 1) // 2
    if kernel_size % 2 == 1:
        center = factor - 1
    else:
        center = factor - 0.5
    og = np.ogrid[:kernel_size, :kernel_size]
    filt = (1 - abs(og[0] - center) / factor) * \
           (1 - abs(og[1] - center) / factor)
    weight = np.zeros(
        (in_channels, out_channels, kernel_size, kernel_size),
        dtype='float32')
    weight[range(in_channels), range(out_channels), :, :] = filt
    return nd.array(weight)

In [13]:
# from matplotlib import pyplot as plt

# x = train_images[0]
# print('Input', x.shape)
# x = x.astype('float32').transpose((2,0,1)).expand_dims(axis=0)/255

# conv_trans = nn.Conv2DTranspose(
#     3, in_channels=3, kernel_size=8, padding=2, strides=4)
# conv_trans.initialize()
# conv_trans(x)
# conv_trans.weight.set_data(bilinear_kernel(3, 3, 8))


# y = conv_trans(x)
# y = y[0].clip(0,1).transpose((1,2,0))
# print('Output', y.shape)

# plt.imshow(y.asnumpy())
# plt.show()

In [14]:
from mxnet import init
net[-1].initialize()
net[-2].initialize()
# conv_trans = net[-1]
# conv_trans.initialize(init=init.Zero())
# net[-2].initialize(init=init.Xavier())

# x = nd.zeros((batch_size, 3, *input_shape))
# net(x)

# shape = conv_trans.weight.data().shape
# conv_trans.weight.set_data(bilinear_kernel(*shape[0:3]))

In [15]:
import sys
sys.path.append('..')
import utils

# loss = gluon.loss.SoftmaxCrossEntropyLoss(axis=1)
# loss = gluon.loss.SigmoidBinaryCrossEntropyLoss()
def my_loss(pre,label):
    print(pre.shape)
    print(pre[0][0])
    print(label.shape)
    print(label)
    return 0
loss = gluon.loss.L2Loss()
# loss = my_loss



ctx = utils.try_all_gpus()
net.collect_params().reset_ctx(ctx)

trainer = gluon.Trainer(net.collect_params(),
                        'NAG', {'learning_rate': 0.1, 'wd':1e-3})

utils.mytrain(train_data, test_data, net, loss,
            trainer, ctx, num_epochs=30)

Epoch 0. Loss: 0.000, Train acc 0.00, Test acc 0.00, Time 4.9 sec
Epoch 1. Loss: 0.000, Train acc 0.00, Test acc 0.00, Time 3.8 sec
Epoch 2. Loss: 0.000, Train acc 0.00, Test acc 0.00, Time 3.9 sec
Epoch 3. Loss: 0.000, Train acc 0.00, Test acc 0.00, Time 3.8 sec
Epoch 4. Loss: 0.000, Train acc 0.00, Test acc 0.00, Time 3.8 sec
Epoch 5. Loss: 0.000, Train acc 0.00, Test acc 0.00, Time 3.9 sec
Epoch 6. Loss: 0.000, Train acc 0.00, Test acc 0.00, Time 3.9 sec
Epoch 7. Loss: 0.000, Train acc 0.00, Test acc 0.00, Time 3.9 sec
Epoch 8. Loss: 0.000, Train acc 0.00, Test acc 0.00, Time 3.9 sec
Epoch 9. Loss: 0.000, Train acc 0.00, Test acc 0.00, Time 3.9 sec
Epoch 10. Loss: 0.000, Train acc 0.00, Test acc 0.00, Time 3.8 sec
Epoch 11. Loss: 0.000, Train acc 0.00, Test acc 0.00, Time 3.8 sec
Epoch 12. Loss: 0.000, Train acc 0.00, Test acc 0.00, Time 4.0 sec
Epoch 13. Loss: 0.000, Train acc 0.00, Test acc 0.00, Time 3.9 sec
Epoch 14. Loss: 0.000, Train acc 0.00, Test acc 0.00, Time 3.8 sec
Epoch

In [20]:
import mxnet as mx
# def predict(im):
# #     mx.random.seed(np.random.randint(1, 1000))
# #     noise = mx.nd.random.normal(0, 1, shape=(640, 640, 1), dtype=np.float32)
# #     data = ((im.astype('float32')+ 5*noise).clip(0, 255) / 255 - rgb_mean) / rgb_std
#     data = normalize_image(im)
# #     data = (im.astype('float32') / 255 - rgb_mean) / rgb_std
#     data = data.transpose((2,0,1)).expand_dims(axis=0)
#     yhat = net(data.as_in_context(ctx[0]))
#     pred = nd.argmax(yhat, axis=1)
#     return pred.reshape((pred.shape[1], pred.shape[2]))

def predict(im):
#     mx.random.seed(np.random.randint(1, 1000))
#     noise = mx.nd.random.normal(0, 1, shape=(640, 640, 1), dtype=np.float32)
#     data = ((im.astype('float32')+ 5*noise).clip(0, 255) / 255 - rgb_mean) / rgb_std
    data = normalize_image(im)
#     data = (im.astype('float32') / 255 - rgb_mean) / rgb_std
    data = data.transpose((2,0,1)).expand_dims(axis=0)
    yhat = net(data.as_in_context(ctx[0]))
    
    return yhat

def label2image(pred):
    x = pred.astype('int32').asnumpy()
    cm = nd.array(colormap).astype('uint8')
    return nd.array(cm[x,:])

In [17]:
test_images,test_labels = read_images(1)

In [18]:
def evaluate_acc(result, label):
    if(len(np.where(result.asnumpy()>0)[1]) ==0):
        return False,False,False,False,False,False,False,False
    result_maxx = np.max(np.where(result.asnumpy()>0)[1])
    result_minx = np.min(np.where(result.asnumpy()>0)[1])
    result_maxy = np.max(np.where(result.asnumpy()>0)[0])
    result_miny = np.min(np.where(result.asnumpy()>0)[0])

    result_centerx = int((result_maxx + result_minx)/2)
    result_centery = int((result_maxy + result_miny)/2)
    
    result_avgx = np.average(np.where(result.asnumpy()>0)[1])
    result_avgy = np.average(np.where(result.asnumpy()>0)[0])

    label_maxx = np.max(np.where(label.asnumpy()>0)[1])
    label_minx = np.min(np.where(label.asnumpy()>0)[1])
    label_maxy = np.max(np.where(label.asnumpy()>0)[0])
    label_miny = np.min(np.where(label.asnumpy()>0)[0])
    
    label_centerx = int((label_maxx + label_minx)/2)
    label_centery = int((label_maxy + label_miny)/2)
    
    lable_avgx = np.average(np.where(label.asnumpy()>0)[1])
    lable_avgy = np.average(np.where(label.asnumpy()>0)[0])
    
    d1 = pow((result_centerx - label_centerx),2) + pow((result_centery - label_centery),2)
    d2 = pow((result_avgx - label_centerx),2) + pow((result_avgy - label_centery),2)
#     print("============================")
#     print(np.where(result.asnumpy()>0))
#     print(result_centerx,label_centerx)
#     print(result_centery,label_centery)
#     print(d1,d2)
    return (d1 < 400, d2 < 400, d1 < 625, d2 < 625, d1 < 900, d2 < 900, d1 < 1600, d2 < 1600)

In [21]:
# n = len(test_images)
n = 10
imgs = []
acc1 = 0
acc2 = 0
acc3 = 0
acc4 = 0
acc5 = 0
acc6 = 0
acc7 = 0
acc8 = 0
# print(test_images[0])
for i in range(n):
    x = test_images[i]
#     print(x.shape)
#     start = time()
    result = predict(x)
    print(result)
    f1,f2,f3,f4,f5,f6,f7,f8 = evaluate_acc(result,test_labels[i])
    acc1 += int(f1)
    acc2 += int(f2)
    acc3 += int(f3)
    acc4 += int(f4)
    acc5 += int(f5)
    acc6 += int(f6)
    acc7 += int(f7)
    acc8 += int(f8)

#     print(type(np.where(result.asnumpy()>0)[0]))
#     print(np.where(test_labels[i].asnumpy()>0))
    pred = label2image(predict(x))
    imgs += [x, pred, label2image(test_labels[i])]
print(acc1/n,"  ", acc2/n,"  ", acc3/n,"  ", acc4/n,"  ", acc5/n,"  ", acc6/n,"  ", acc7/n,"  ", acc8/n)
utils.show_images(imgs, nrows=n, ncols=3, figsize=(6,20))


MXNetError: [22:56:34] src/storage/./pooled_storage_manager.h:108: cudaMalloc failed: out of memory

Stack trace returned 10 entries:
[bt] (0) /home/charming/miniconda3/envs/gluon/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x32173a) [0x7f664e6a473a]
[bt] (1) /home/charming/miniconda3/envs/gluon/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x321d61) [0x7f664e6a4d61]
[bt] (2) /home/charming/miniconda3/envs/gluon/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x297d083) [0x7f6650d00083]
[bt] (3) /home/charming/miniconda3/envs/gluon/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2981935) [0x7f6650d04935]
[bt] (4) /home/charming/miniconda3/envs/gluon/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x26ab94a) [0x7f6650a2e94a]
[bt] (5) /home/charming/miniconda3/envs/gluon/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x26bfc5a) [0x7f6650a42c5a]
[bt] (6) /home/charming/miniconda3/envs/gluon/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x26bfd8b) [0x7f6650a42d8b]
[bt] (7) /home/charmi

In [ ]:
# test_images,test_labels = read_images(18,2)

In [ ]:
# n = 10
# imgs = []
# for i in range(n):
#     x = test_images[i]
#     imgs += [x, label2image(test_labels[i]), label2image(test_labels[i])]
# # 
# utils.show_images(imgs, nrows=n, ncols=3, figsize=(6,10))


In [ ]:
for i in range(0):
    print(i)